<a href="https://colab.research.google.com/github/meifa00/BMCS2114_European_Soccer/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preprocessing**

In [15]:
!pip install ipynb

In [16]:
from ipynb.fs.full.Data_Understanding import *

ModuleNotFoundError: No module named 'ipynb.fs.full.Data_Understanding'

## **Data Cleaning**

### **Country and League Table**

In [ ]:
# rename the name column in 'League' table
league_df.rename(columns={'name':'league_name'},inplace=True)

Merging the two tables to produce a new DataFrame.

In [ ]:
country_league_df = country_df.merge(league_df,left_on='id',right_on='id')

country_league_df.drop(columns='country_id',inplace=True)

country_league_df.rename(columns={'id':'country_id','name':'country_name'},inplace=True)

In [ ]:
# new DataFrame
country_league_df

### **Match Table**

**Missing Data**

In [ ]:
# drop columns with null values
match_df.dropna(axis=1,inplace=True)

**Data Type Issues**

In [ ]:
# 'date' column to Datetime datatype
match_df['date'] = pd.to_datetime(match_df['date'])

Add two additional columns to the DataFrame with the starting and completion year of a season.

In [ ]:
begin_season = match_df['season'].apply(lambda x:x.split('/')[0])
finish_season = match_df['season'].apply(lambda x:x.split('/')[1])

match_df['begin_season'] = begin_season
match_df['finish_season'] = finish_season

# drop the season column which is not required anymore
# match_df.drop(columns=['season'],inplace=True,axis=1)

Create an additional column in the DataFrame with the year when the match was played.

In [ ]:
# create new column
match_df['match_year'] = match_df['date'].dt.year

Merge the 'Match' and 'CountryLeague' table generates a new table.

In [ ]:
# merge tables
match_df_v2 = match_df.merge(country_league_df,left_on='country_id',right_on='country_id')

In [ ]:
# remove country_id as it is same as league_id are same
match_df_v2.drop(['country_id'],axis=1,inplace=True)
match_df_v2.head()

In [ ]:
# make the two new columns as int
match_df_v2['begin_season'] = match_df_v2['begin_season'].astype('int64')
match_df_v2['finish_season'] = match_df_v2['finish_season'].astype('int64')

In [ ]:
match_df_v2.info()

In [ ]:
# save the cleaned match DataFrame for further reference
match_df_v2.to_csv('/content/drive/MyDrive/Colab Notebooks/European Soccer/match_merged.csv',index=False)

### **Player Table**

**Data Type Issues**

In [ ]:
# 'birthday' column to Datetime datatype
player_df['birthday'] = pd.to_datetime(player_df['birthday'])

In [ ]:
# change column weight to float.
player_df.weight = player_df.weight.astype('float')

In [ ]:
player_df.info()

### **Player Attributes Table**

**Missing Data**

In [ ]:
numeric_columns = player_attributes_df.select_dtypes(include=['number']).columns
categorical_columns = player_attributes_df.select_dtypes(include=['object']).columns

In [ ]:
player_attributes_df[numeric_columns] = player_attributes_df[numeric_columns].fillna(player_attributes_df[numeric_columns].mean())

In [ ]:
for col in categorical_columns:
    mode_val = player_attributes_df[col].mode()[0]
    player_attributes_df[col].fillna(mode_val, inplace=True)

In [ ]:
player_attributes_df.isnull().sum()

**Data Type Issues**

In [ ]:
# change 'date' type of date column
player_attributes_df['date'] = pd.to_datetime(player_attributes_df['date'])

In [ ]:
player_attributes_df.info()

Create a new table by merging the 'Player' and 'Player Attributes' tables.

In [ ]:
player_with_attributes_df = player_df.merge(player_attributes_df,left_on='player_api_id',right_on='player_api_id')

In [ ]:
# rename few columns
player_with_attributes_df.rename(columns={'id_x':'id','player_fifa_api_id_x':'player_fifa_api_id'},inplace=True)

# drop the columns that have got merged twice
player_with_attributes_df.drop(['player_fifa_api_id_y','id_y'],axis=1,inplace=True)

In [ ]:
# save the cleaned player with atrributes DataFrame for further reference
player_with_attributes_df.to_csv('/content/drive/MyDrive/Colab Notebooks/European Soccer/player_att_merged.csv',index=False)

### **Team Table**

In [ ]:
# drop the unwanted columns
team_df.drop(['id','team_fifa_api_id'],axis=1,inplace=True)

Combine the 'Team' table and 'Match' table.

In [ ]:
# add the home team column
match_df_v2 = match_df_v2.merge(team_df,left_on='home_team_api_id',right_on='team_api_id')
match_df_v2.rename(columns={'team_long_name':'home_team_name'},inplace=True)
match_df_v2.drop(['team_api_id','team_short_name'],axis=1,inplace=True)

# add the away team column
match_df_v2 = match_df_v2.merge(team_df,left_on='away_team_api_id',right_on='team_api_id')
match_df_v2.rename(columns={'team_long_name':'away_team_name'},inplace=True)
match_df_v2.drop(['team_api_id','team_short_name'],axis=1,inplace=True)

Generate a new column with the name of the victorious team for each individual match.

In [ ]:
# function to find and return the name of the team
def win(i):
    home_team_name = i[0]
    home_team_score = i[1]
    away_team_name = i[2]
    away_team_score = i[3]

    if home_team_score > away_team_score:
        return home_team_name
    elif home_team_score < away_team_score:
        return away_team_name
    else:
        return 'Tie'

In [ ]:
# apply the function to the DataFrame
match_df_v2['winning_team']  = match_df_v2[['home_team_name', 'home_team_goal', 'away_team_name', 'away_team_goal']].apply(win, axis=1)

In [ ]:
match_df_v2.head()

In [ ]:
# update the 'Match' DataFrame
match_df_v2.to_csv('/content/drive/MyDrive/Colab Notebooks/European Soccer/match_merged.csv',index=False)

### **Team Attributes Table**

**Missing Data**

In [ ]:
team_attributes_df.drop(columns=['buildUpPlayDribbling'], inplace=True)

In [ ]:
team_attributes_df.isnull().sum()

**Data Type Issues**

In [ ]:
# change data type
team_attributes_df['date'] = pd.to_datetime(team_attributes_df['date'])

# drop unwanted columns
team_attributes_df.drop(['team_fifa_api_id'],axis=1,inplace=True)

In [ ]:
team_attributes_df.info()

Create a new column to indicate the data collection date.

In [ ]:
team_attributes_df['year'] = team_attributes_df['date'].dt.year

Combine the 'Team' table with the 'Team Attributes' table.

In [ ]:
team_attributes_df_v2 = team_attributes_df.merge(team_df,left_on='team_api_id',right_on='team_api_id')

In [ ]:
team_attributes_df_v2.head()

In [ ]:
# save the cleaned team with atrributes DataFrame for further reference
team_attributes_df_v2.to_csv('/content/drive/MyDrive/Colab Notebooks/European Soccer/team_att_merged.csv',index=False)